In [3]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import cvxpy as cp

In [8]:
#Problem 4

x = cp.Variable(2, integer = True)

obj_func = -4*x[0] + -8*x[1]

constraints = []
constraints.append(2*x[0] + 2*x[1] <= 19)
constraints.append(-2*x[0] + 2*x[1] <= 3)
constraints.append(x[0] >= 0)
constraints.append(x[1] >= 0)

problem = cp.Problem(cp.Minimize(obj_func), constraints)

problem.solve(solver=cp.GUROBI, verbose = True)

print("obj_func =")
print(obj_func.value)
print("x =")
print(x.value)

                                     CVXPY                                     
                                     v1.5.3                                    
(CVXPY) Oct 25 05:26:54 PM: Your problem has 2 variables, 4 constraints, and 0 parameters.
(CVXPY) Oct 25 05:26:54 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Oct 25 05:26:54 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Oct 25 05:26:54 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Oct 25 05:26:54 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Oct 25 05:26:54 PM: Compiling problem (target solver=GUROBI).
(CV

In [5]:
#Problem5

# Define the wine content for each bottle
wine_content = [1] * 7 + [0.5] * 7 + [0] * 7

# Define binary variables x[i, j] where i is the bottle index and j is the person index (A, B, C)
x = cp.Variable((21, 3), boolean=True)

# Constraints
constraints = []

# Each bottle must be assigned to exactly one person
for i in range(21):
    constraints.append(x[i, 0] + x[i, 1] + x[i, 2] == 1)

# Each person must receive exactly 7 bottles
for j in range(3):
    constraints.append(cp.sum(x[:, j]) == 7)

# The total wine quantity must be equal for all individuals
total_wine_A = cp.sum(cp.multiply(wine_content, x[:, 0]))
total_wine_B = cp.sum(cp.multiply(wine_content, x[:, 1]))
total_wine_C = cp.sum(cp.multiply(wine_content, x[:, 2]))
constraints.append(total_wine_A == total_wine_B)
constraints.append(total_wine_B == total_wine_C)

# Define a dummy objective function
objective = cp.Minimize(0)

# Solve the problem
prob = cp.Problem(objective, constraints)
prob.solve()

# Output results
if prob.status == cp.OPTIMAL or prob.status == cp.FEASIBLE:
    assignment = x.value.round().astype(int)
    for j, person in enumerate(['A', 'B', 'C']):
        bottles = [i + 1 for i in range(21) if assignment[i, j] == 1]
        print(f"Person {person} receives bottles: {bottles}")
else:
    print("No feasible solution found.")

Person A receives bottles: [3, 4, 7, 13, 15, 16, 17]
Person B receives bottles: [2, 5, 8, 12, 14, 19, 21]
Person C receives bottles: [1, 6, 9, 10, 11, 18, 20]


In [11]:
#Problem6

# Parameters
U_values = [500, 3000]
U = cp.Parameter(nonneg=True)

# Decision variables
x_A = cp.Variable(boolean=True)
x_B = cp.Variable(boolean=True)
x_C = cp.Variable(boolean=True)

# Cost expressions for each company
C_A = x_A * (10 + cp.minimum(U, 1000) * 0.05 + cp.maximum(U - 1000, 0) * 0.04)
C_B = x_B * (20 + U * 0.04)
C_C = x_C * (25 + cp.minimum(U, 1000) * 0.05 + cp.maximum(U - 1000, 0) * 0.035)

# Total cost
total_cost = C_A + C_B + C_C

# Objective: Minimize total cost
objective = cp.Minimize(total_cost)

# Constraints
constraints = [
    x_A + x_B + x_C == 1,  # Only one company can be selected
    x_A >= 0, x_B >= 0, x_C >= 0
]

# Solve for each value of U
for U_val in U_values:
    U.value = U_val
    problem = cp.Problem(objective, constraints)
    problem.solve()
    
    # Output the results
    selected_company = "Company A" if x_A.value > 0.5 else "Company B" if x_B.value > 0.5 else "Company C"
    print(f"For U = {U_val} minutes:")
    print(f"  Minimum monthly cost: ${total_cost.value:.2f}")
    print(f"  Selected company: {selected_company}")

For U = 500 minutes:
  Minimum monthly cost: $35.00
  Selected company: Company A
For U = 3000 minutes:
  Minimum monthly cost: $140.00
  Selected company: Company A


C:\Users\w6614\AppData\Roaming\Python\Python39\site-packages\cvxpy\reductions\solvers\solving_chain.py:237: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Disciplined Parametrized Programming, at https://www.cvxpy.org/tutorial/dpp/index.html
  warnings.warn(DPP_ERROR_MSG)
